## Overview

This notebook will provide an introduction to code generation using Codey models. Codey for Code Generation (code-bison) is a foundation model that can generate code based on a natural language description. It can be used to create functions, web pages, unit tests, and other types of code. Codey for Code Generation is supported by the code generation Codey APIs, which are part of the PaLM API family.

## Objectives
In this tutorial, we will learn.

* Setting up the environment for code generation models
* Writing basic prompts for codey
* Various prompt design patterns:
    * Problem Statement Template
    * SQL Metadata & Performace
    * Code Optimization
    * Chain of Thought
    * Few Shot Prompts
    * DevOps Templates
    * Web Templates
* Best Practices with Code Generation


## Available models

The Vertex AI Codey API currently supports three models:

* `code-bison@002`: A model fine-tuned to generate code based on a natural language description of the desired code. For example, it can generate a unit test for a function.

* `code-gecko@002`: A model fine-tuned to suggest code completion based on the context in code that's written.

* `codechat-bison@002`: A model fine-tuned for chatbot conversations that help with code-related questions.

We can find more information about the properties of these foundational models in the [Generative AI Studio documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#foundation_models).


#### Code generation with code-bison@002

The code generation model (Codey) from PaLM API that we will use in this notebook is `code-bison@002`. It is fine-tuned to follow natural language instructions to generate required code and is suitable for a variety of coding tasks, such as:

* writing functions
* writing classes
* web-apges
* unit tests
* docstrings
* code translations, and many more use-cases.

Currently it supports the following languages:

* C++
* C#
* Go
* GoogleSQL
* Java
* JavaScript
* Kotlin
* PHP
* Python
* Ruby
* Rust
* Scala
* Swift
* TypeScript

You can find our more details [here](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview).

In [1]:
# import libraries
import os
import vertexai
from IPython.display import Markdown, display
from google.oauth2 import service_account
from dotenv import load_dotenv
from vertexai.language_models import CodeGenerationModel

In [2]:
# initiate service account (authentication)
json_path = '../llm-ai.json' # replace with your own service account
credentials = service_account.Credentials.from_service_account_file(json_path)

In [3]:
# start Vertex AI
load_dotenv()
vertexai.init(project=os.environ["PROJECT_ID"], # replace with your own project
              credentials=credentials)

In [4]:
# load the model
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@002")

**Model parameters for `code-bison`**

We can customize how the PaLM API code generation behaves in response to our prompt by using the following parameters for code-bison@002:

* `prefix`: it represents the beginning of a piece of meaningful programming code or a natural language prompt that describes code to be generated.
* `temperature`: higher means more "creative" code responses. range: (0.0 - 1.0, default 0).
* `max_output_tokens`: sets the max number of tokens in the output. range: (1 - 2048, default 2048)

### 1. Hello Codey

In [6]:
prefix = "write a python function to do similarity check between two strings"

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```python
def similarity_check(str1, str2):
  """
  This function checks the similarity between two strings.

  Args:
    str1 (str): The first string.
    str2 (str): The second string.

  Returns:
    float: A value between 0 and 1, where 0 indicates no similarity and 1 indicates perfect similarity.
  """

  # Convert the strings to lowercase.
  str1 = str1.lower()
  str2 = str2.lower()

  # Find the length of the longest string.
  max_len = max(len(str1), len(str2))

  # Find the number of common characters.
  common_chars = 0
  for i in range(max_len):
    if str1[i] == str2[i]:
      common_chars += 1

  # Calculate the similarity score.
  similarity_score = common_chars / max_len

  return similarity_score
```

### 2. Try out your own prompt

Some examples:

* write Go program to extract ip addresses from the text file
* write Java program that can extract pincodes from addresses
* write a standard SQL function that strips all non-alphabet characters from the string and encodes it to utf-8

In [7]:
prefix = """write a python code that can do predict sales from sales data. The code should have,
            linear regression model. \
          """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024).text

# you may use print instead of display
display(
    Markdown(response)
)

```python
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the sales data
data = pd.read_csv('sales_data.csv')

# Create the features and target variables
features = data[['TV', 'Radio', 'Newspaper']]
target = data['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

# Create the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)

# Evaluate the model's performance
print('MAE:', mean_absolute_error(y_test, predictions))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))
```

### 3. Prompt templates

Prompt templates are useful if we have found a good way to structure our prompt that we can re-use. This can be also be helpful in limiting the open-endedness of freeform prompts. There are many ways to implement prompt templates, and below is just one example using `f-strings`. This way we can structure the prompts as per the expected funcationality of the code.

In [8]:
language = "python"
prefix = f"""Write a {language} function that can input unsorted array or list and return the sorted array or list.
             it should not use any pre-built function or library.
              """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024).text

# you may use print instead of display
display(
    Markdown(response)
)

```python
def bubble_sort(array):
  """
  Sorts an array using the bubble sort algorithm.

  Args:
    array: The array to be sorted.

  Returns:
    The sorted array.
  """

  # Iterate over the array, comparing each element to the next.
  for i in range(len(array) - 1):
    # If the current element is greater than the next element, swap them.
    if array[i] > array[i + 1]:
      array[i], array[i + 1] = array[i + 1], array[i]

  # Repeat the process until the array is sorted.
  if not is_sorted(array):
    return bubble_sort(array)

  return array


def is_sorted(array):
  """
  Checks if an array is sorted.

  Args:
    array: The array to be checked.

  Returns:
    True if the array is sorted, False otherwise.
  """

  for i in range(len(array) - 1):
    if array[i] > array[i + 1]:
      return False

  return True
```

In [9]:
language = "python"
file_format = "json"
extract_info = "names"
requirements = """
              - the name should be start with capital letters.
              - There should be no duplicate names in the final list.
              """

prefix = f"""
        Create a {language} to parse {file_format} and extract {extract_info} 
        with the following requirements: {requirements}. 
        """

response = code_generation_model.predict(prefix=prefix, max_output_tokens=1024).text

# you may use print instead of display
display(
    Markdown(response)
)

```python
import json

# Load the JSON data from a file
with open('data.json') as f:
    data = json.load(f)

# Create a set to store the unique names
names = set()

# Iterate over the data and extract the names
for person in data:
    name = person['name']

    # Check if the name starts with a capital letter
    if name[0].isupper():
        # Add the name to the set
        names.add(name)

# Print the unique names
print(names)
```

### 4. Prompt Design Patterns

**Problem-statement Template**

In the problem-statement template, we can leverage prompt template ideas and pass our specific problem-statement without focusing on the language. The language can be a separate parameter. We can also pass the input-output example to ensure that generation follows the test case. Here are examples of different languages; we can see the model's capability to generate many supported languages.

* **C**

In [10]:
language = "c"
problem_statement = "find the smallest element in an unordered list"

prefix = "write a " + language + " function to " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```c
#include <stdio.h>

int find_smallest(int arr[], int n) {
    int smallest = arr[0];
    for (int i = 1; i < n; i++) {
        if (arr[i] < smallest) {
            smallest = arr[i];
        }
    }
    return smallest;
}

int main() {
    int arr[] = {10, 20, 5, 30, 15};
    int n = sizeof(arr) / sizeof(arr[0]);
    int smallest = find_smallest(arr, n);
    printf("The smallest element in the list is: %d\n", smallest);
    return 0;
}
```

* **C++**

In [11]:
language = "cpp"
problem_statement = """Sort an array in one swap whose two elements are swapped and rest are in sorted order \
                      for example: \
                      input: {1, 5, 3, 7, 9}
                      output: {1,3,5,7,9}
                    """

prefix = "write a " + language + " function to " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```c++
#include <iostream>
#include <vector>

using namespace std;

void oneSwapSort(vector<int>& arr) {
  int n = arr.size();

  // Find the first element that is not in sorted order
  int i = 0;
  while (i < n - 1 && arr[i] <= arr[i + 1]) {
    i++;
  }

  // If the array is already sorted, return
  if (i == n - 1) {
    return;
  }

  // Find the second element that is not in sorted order
  int j = n - 1;
  while (j > 0 && arr[j] >= arr[j - 1]) {
    j--;
  }

  // Swap the two elements
  swap(arr[i], arr[j]);
}

int main() {
  vector<int> arr = {1, 5, 3, 7, 9};

  oneSwapSort(arr);

  for (int i = 0; i < arr.size(); i++) {
    cout << arr[i] << " ";
  }

  cout << endl;

  return 0;
}
```

* **TypeScript**

In [12]:
language = "typescript"
problem_statement = """that takes a timestamp and return as yyyy-mm-dd in local time.
                      print the outputs with react component
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```typescript
// Function to convert a timestamp to a yyyy-mm-dd string in local time
const formatDate = (timestamp: number) => {
  const date = new Date(timestamp);
  const year = date.getFullYear();
  const month = date.getMonth() + 1; // Months are zero-indexed
  const day = date.getDate();
  return `${year}-${month}-${day}`;
};

// React component to display the formatted date
const DateComponent = () => {
  const timestamp = 1654636800000; // Example timestamp
  const formattedDate = formatDate(timestamp);

  return (
    <div>
      <h1>Formatted Date: {formattedDate}</h1>
    </div>
  );
};

export default DateComponent;
```

* **Elixir**

In [13]:
language = "elixir"
problem_statement = """print the first non-repeated character from a string.
                      take example of 'Mississippi' and 'hello' as example.
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```elixir
defmodule NonRepeatedCharacter do
  def find_first_non_repeated_character(string) do
    string
    |> String.codepoints()
    |> Enum.reduce(%{}, fn char, acc ->
      case Map.get(acc, char) do
        nil -> Map.put(acc, char, 1)
        _ -> Map.put(acc, char, Map.get(acc, char) + 1)
      end
    end)
    |> Enum.find(fn {_, count} -> count == 1 end)
    |> elem(0)
  end
end

IO.puts NonRepeatedCharacter.find_first_non_repeated_character("Mississippi")
IO.puts NonRepeatedCharacter.find_first_non_repeated_character("hello")
```

* **R**

In [14]:
language = "r"
problem_statement = """reverse an array in place. Take input of array.Take input of array and output the reversed array.
                       For example: [1,2,3,4,5] -> [5,4,3,2,1]
                    """

prefix = "write a " + language + " program " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```r
# Function to reverse an array in place
reverse_array <- function(arr) {
  # Get the length of the array
  n <- length(arr)
  
  # Iterate over the array from the beginning to the middle
  for (i in 1:(n/2)) {
    # Swap the elements at indices i and n-i+1
    temp <- arr[i]
    arr[i] <- arr[n-i+1]
    arr[n-i+1] <- temp
  }
  
  # Return the reversed array
  return(arr)
}

# Take input of the array
arr <- as.numeric(unlist(strsplit(readline(), " ")))

# Reverse the array
reversed_arr <- reverse_array(arr)

# Print the reversed array
print(reversed_arr)
```

* **Ruby**

In [15]:
language = "Ruby"
problem_statement = """ to remove duplicate elements from given array.
                    """

prefix = "write a " + language + " program " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```ruby
# This Ruby program removes duplicate elements from a given array.

# Define a method to remove duplicate elements from an array.
def remove_duplicates(array)
  # Create a new array to store the unique elements.
  unique_array = []

  # Iterate over the original array.
  array.each do |element|
    # Check if the element is already in the unique array.
    if !unique_array.include?(element)
      # If the element is not in the unique array, add it.
      unique_array << element
    end
  end

  # Return the unique array.
  unique_array
end

# Get the input array from the user.
puts "Enter the elements of the array, separated by spaces:"
input_array = gets.chomp.split(" ")

# Convert the input array to an integer array.
int_array = input_array.map(&:to_i)

# Remove duplicate elements from the array.
unique_array = remove_duplicates(int_array)

# Print the unique array.
puts "The unique elements of the array are:"
puts unique_array
```

* **Go (Golang)**

In [16]:
language = "Go"
problem_statement = """that can extract ipv4 addresses from the each line in the log file. use fmt and regexp package.
                        input:
                        03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0
                        output:
                        129.1.1.1
                        \n\n
                    """

prefix = "write a " + language + " function " + problem_statement

response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```go
package main

import (
    "fmt"
    "regexp"
)

func main() {
    // Sample log line
    logLine := "03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0"

    // Compile the regular expression to match IPv4 addresses
    ipv4Regex := regexp.MustCompile(`\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b`)

    // Find all matches of the regular expression in the log line
    matches := ipv4Regex.FindAllString(logLine, -1)

    // Print the extracted IPv4 addresses
    for _, match := range matches {
        fmt.Println(match)
    }
}
```

### 5. SQL Metadata & Performance

In the SQL prompt template, we can define multiple criteria like problem statement, table metadata, styling of the code, and performance expectations. Passing the table metadata is crucial since it allows the model to generate consistent code that follows the structure of the metadata. Ensure that your problem statement is very clear, concise, and contains all relevant context.

* **SQL**

In [17]:
problem_statement = """
                    You are a developer working on an e-commerce platform.
                    The marketing team has requested a report on the total number of orders and the average order \
                    value for each product category for the past month.
                    Your task is to generate a SQL queries to retrieve the total number of orders and the average order \
                    value for each product category for the orders placed in the:
                    1) past month,
                    2) given data range,
                    3) end of each month for given year,
                    4) christmas and new year's eve.
                    """
table_metadata = """
                 - **Orders:**
                    - `OrderID` (integer)
                    - `ProductID` (integer)
                    - `ProductName` (string)
                    - `Category` (string)
                    - `OrderDate` (date)
                    - `OrderAmount` (decimal)
                """
code_style = """
            Write a SQL query that follows best practices, is readable, and well-commented.
             """

performance_requirement = """
                          Optimize the query for performance considering the potential size of the "Orders" table.
                          Consider using appropriate indexing if necessary.
                          """

prefix = f""" Solve the following: {problem_statement}. The given table metadata is : {table_metadata} .
          Follow the following code style:{code_style} . The following performance requirement is: {performance_requirement} .
          """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```sql
-- Calculate the total number of orders and average order value for each product category in the past month

SELECT
    p.Category,
    COUNT(o.OrderID) AS TotalOrders,
    AVG(o.OrderAmount) AS AverageOrderValue
FROM
    Orders o
JOIN
    Products p ON o.ProductID = p.ProductID
WHERE
    o.OrderDate BETWEEN DATE('now', '-1 month') AND DATE('now') -- Filter orders within the past month
GROUP BY
    p.Category;


-- Calculate the total number of orders and average order value for each product category for a given date range

SELECT
    p.Category,
    COUNT(o.OrderID) AS TotalOrders,
    AVG(o.OrderAmount) AS AverageOrderValue
FROM
    Orders o
JOIN
    Products p ON o.ProductID = p.ProductID
WHERE
    o.OrderDate BETWEEN '2023-01-01' AND '2023-01-31' -- Replace with your desired date range
GROUP BY
    p.Category;


-- Calculate the total number of orders and average order value for each product category at the end of each month for a given year

SELECT
    strftime('%Y-%m', o.OrderDate) AS OrderMonth, -- Extract the year and month from the OrderDate
    p.Category,
    COUNT(o.OrderID) AS TotalOrders,
    AVG(o.OrderAmount) AS AverageOrderValue
FROM
    Orders o
JOIN
    Products p ON o.ProductID = p.ProductID
WHERE
    o.OrderDate BETWEEN '2023-01-01' AND '2023-12-31' -- Replace with your desired year
GROUP BY
    OrderMonth,
    p.Category;


-- Calculate the total number of orders and average order value for each product category during Christmas and New Year's Eve

SELECT
    p.Category,
    COUNT(o.OrderID) AS TotalOrders,
    AVG(o.OrderAmount) AS AverageOrderValue
FROM
    Orders o
JOIN
    Products p ON o.ProductID = p.ProductID
WHERE
    o.OrderDate BETWEEN '2022-12-24' AND '2023-01-01' -- Filter orders between Christmas and New Year's Eve
GROUP BY
    p.Category;
```

* **BigQuery**

In [20]:
metadata = """
          A table of customer data, with the following columns:

          customer_id: The unique identifier for the customer.
          first_name: The customer's first name.
          last_name: The customer's last name.
          email: The customer's email address.
          phone_number: The customer's phone number.
          country: The customer's country of residence.
          order_history: A JSON object containing the customer's order history, including the following information for each order:
          order_id: The unique identifier for the order.
          order_date: The date the order was placed.
          order_total: The total amount of the order.
          order_items: A list of the items ordered, including the following information for each item:
          item_id: The unique identifier for the item.
          item_name: The name of the item.
          item_quantity: The quantity of the item ordered.
          item_price: The price of the item.

            """
language = "BigQuery"
problem = """solve following queries: \n
            - Total number of orders placed. \n
            - Total amount of money spent on orders. \n
            - Average order value. \n
            - Most popular item ordered (by quantity). \n
            - Most recent order placed. \n
          """
additional_requirment = """
            - The query should be efficient and scalable, as the customer table may contain millions of rows. \n
            - The query should be easy to read and maintain.
            """
prefix = f"""Write a {language} query to {problem}.
          use this as the table metadata: {metadata}.
          Here are some additional requirement for the query: {additional_requirment}.
          Generate each query as a separate query seperated with a comment.
              """
response = code_generation_model.predict(prefix=prefix, max_output_tokens=2000).text

# you may use print instead of display
display(
    Markdown(response)
)

```sql
-- Total number of orders placed
SELECT COUNT(DISTINCT order_id) AS total_orders
FROM customer_data, UNNEST(order_history) AS order_info;
```

### 6. Code Optimization

We can define our problem statement's specific optimization requirements in the code optimization prompt. The codey models are great at following particular instructions and generating code that can meet user-specific conditions. We can experiment below with three languages and see how to give specific instructions on data structures, algorithmic complexity, and maintainability.

* **Heskell**

In [21]:
sample_input = "A list of integers, e.g. [1, 2, 3, 4, 5]"
language = "Haskell"
problem_statement = f"a {language} program that calculates the sum of all the integers in the list."
additional_requirement = """
                      - The function should be efficient and recursive.
                      - The function should be polymorphic, so that it can be used to sum lists of any type of number.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix, max_output_tokens=1000).text

# you may use print instead of display
display(
    Markdown(response)
)

```haskell
-- Recursive function to calculate the sum of a list of numbers
sumList :: Num a => [a] -> a
sumList [] = 0
sumList (x:xs) = x + sumList xs

-- Example use case
main :: IO ()
main = do
  let list = [1, 2, 3, 4, 5]
  print $ sumList list -- Output: 15
```

* **Java**

In [22]:
sample_input = """ "(1 + 2) * 3"
              """
language = "Java"
problem_statement = f"a {language} program that evaluates the mathematical expression and prints the result to the console."
additional_requirement = """
                      - The program should handle all valid mathematical expressions, including those with parentheses, operators, and variables.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Use {sample_input} to test the generated code.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```java
import java.util.Scanner;

public class ExpressionEvaluator {

    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);

        // Get the mathematical expression from the user.
        System.out.println("Enter a mathematical expression: ");
        String expression = scanner.nextLine();

        // Evaluate the expression and print the result.
        double result = evaluateExpression(expression);
        System.out.println("The result of the expression is: " + result);
    }

    public static double evaluateExpression(String expression) {
        // Create a stack to store the operands and operators.
        Stack<Double> operands = new Stack<>();
        Stack<Character> operators = new Stack<>();

        // Tokenize the expression.
        String[] tokens = expression.split(" ");

        // Iterate over the tokens.
        for (String token : tokens) {
            // If the token is an operand, push it onto the operands stack.
            if (isOperand(token)) {
                operands.push(Double.parseDouble(token));
            }
            // If the token is an operator, push it onto the operators stack.
            else if (isOperator(token)) {
                operators.push(token.charAt(0));
            }
            // If the token is a left parenthesis, push it onto the operators stack.
            else if (token.equals("(")) {
                operators.push('(');
            }
            // If the token is a right parenthesis, pop operators from the operators stack and apply them to the operands stack until a left parenthesis is encountered.
            else if (token.equals(")")) {
                while (!operators.isEmpty() && operators.peek() != '(') {
                    double operand2 = operands.pop();
                    double operand1 = operands.pop();
                    char operator = operators.pop();
                    double result = applyOperator(operator, operand1, operand2);
                    operands.push(result);
                }

                // Pop the left parenthesis from the operators stack.
                operators.pop();
            }
        }

        // While there are still operators on the operators stack, pop them and apply them to the operands stack.
        while (!operators.isEmpty()) {
            double operand2 = operands.pop();
            double operand1 = operands.pop();
            char operator = operators.pop();
            double result = applyOperator(operator, operand1, operand2);
            operands.push(result);
        }

        // The last element on the operands stack is the result of the expression.
        return operands.pop();
    }

    public static boolean isOperand(String token) {
        try {
            Double.parseDouble(token);
            return true;
        } catch (NumberFormatException e) {
            return false;
        }
    }

    public static boolean isOperator(String token) {
        return token.equals("+") || token.equals("-") || token.equals("*") || token.equals("/");
    }

    public static double applyOperator(char operator, double operand1, double operand2) {
        switch (operator) {
            case '+':
                return operand1 + operand2;
            case '-':
                return operand1 - operand2;
            case '*':
                return operand1 * operand2;
            case '/':
                return operand1 / operand2;
            default:
                throw new IllegalArgumentException("Invalid operator: " + operator);
        }
    }
}
```

* **JavaScript**

In [25]:
sample_input = """
              A JSON object containing a list of products, each product with the following properties:
              id: The unique identifier for the product.
              name: The name of the product.
              price: The price of the product.
              quantity: The quantity of the product in stock.
              """
language = "JavaScript"
problem_statement = f"""a {language} function that takes the JSON object as input and returns a new JSON object containing the following properties:
                      total_price: The total price of all the products in the list.
                      average_price: The average price of all the products in the list.
                      most_expensive_product: The most expensive product in the list.
                      least_expensive_product: The least expensive product in the list.
                      out_of_stock_products: A list of all the products that are out of stock.
                    """
additional_requirement = """
                      - The function should be efficient and scalable, as the JSON object may contain millions of products.
                      - The function should be easy to read and maintain.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```javascript
// Function to calculate product statistics
function calculateProductStatistics(products) {
  // Initialize variables
  let total_price = 0;
  let average_price = 0;
  let most_expensive_product = null;
  let least_expensive_product = null;
  let out_of_stock_products = [];

  // Iterate over the products
  for (let i = 0; i < products.length; i++) {
    const product = products[i];

    // Calculate total price
    total_price += product.price * product.quantity;

    // Calculate average price
    average_price = total_price / products.length;

    // Find most expensive product
    if (most_expensive_product === null || product.price > most_expensive_product.price) {
      most_expensive_product = product;
    }

    // Find least expensive product
    if (least_expensive_product === null || product.price < least_expensive_product.price) {
      least_expensive_product = product;
    }

    // Add out of stock products to the list
    if (product.quantity === 0) {
      out_of_stock_products.push(product);
    }
  }

  // Return the product statistics
  return {
    total_price: total_price,
    average_price: average_price,
    most_expensive_product: most_expensive_product,
    least_expensive_product: least_expensive_product,
    out_of_stock_products: out_of_stock_products,
  };
}

// Example use case
const products = [
  {
    id: 1,
    name: "Product 1",
    price: 10,
    quantity: 5,
  },
  {
    id: 2,
    name: "Product 2",
    price: 20,
    quantity: 3,
  },
  {
    id: 3,
    name: "Product 3",
    price: 30,
    quantity: 0,
  },
];

const productStatistics = calculateProductStatistics(products);

console.log("Total Price:", productStatistics.total_price);
console.log("Average Price:", productStatistics.average_price);
console.log("Most Expensive Product:", productStatistics.most_expensive_product);
console.log("Least Expensive Product:", productStatistics.least_expensive_product);
console.log("Out of Stock Products:", productStatistics.out_of_stock_products);
```

### 7. Chain of Though

`Chain-of-thought` _(CoT)_ prompting is a technique for guiding large language models (LLMs) to generate code by breaking down the task into a series of intermediate natural language reasoning steps. This approach has been shown to improve the quality of generated code compared to traditional prompting methods.

To use `CoT` prompting for code generation, we first need to provide the LLM with a natural language description of the task we want to accomplish. The LLM will then generate a chain of thought, a sequence of natural language steps describing how to solve the task. Finally, the LLM will use the chain of thought to generate the code.

Here is an example of using CoT prompting to generate a `C++` and `Java` function for specific use cases.

* **C++**

In [26]:
language = "C++"
sample_input = """
                17/06/09 20:10:41 INFO slf4j.Slf4jLogger: Slf4jLogger started
                17/06/09 20:10:41 INFO Remoting: Starting remoting
                17/06/09 20:10:41 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkExecutorActorSystem@mesos-slave-07:55904]
                17/06/09 20:10:41 INFO util.Utils: Successfully started service 'sparkExecutorActorSystem' on port 55904.
              """
additional_requirement = """
                      - It should not use regex to find the given line
                      - the solution should be scalable and should scale linearly with additional data
                      - the output should print a flag and port number both.
                      - All the variable should be properly in scope and should be decalred only once.
                      - The varibles should have scope to be called in the main() function.
                      - The code should be easy to read and maintain
                      - The code should have proper typehints and comments
                      """

prefix = f"""
        Prompt 1: What is the problem we are trying to solve?

        Identify the status of the sparkExecutorActorSystem service in network log and output True or False along with the port, if True.

        Prompt 2: What is the language you want to use to solve this problem?
        {language}

        Prompt 3: What are the inputs and outputs of the function?

        Input: Network log
        Output: Boolean value indicating the status of the sparkExecutorActorSystem service and the port, if True

        Prompt 4: What are the steps involved in identifying the status of the sparkExecutorActorSystem service in network log?

        Split the network log into lines.
        Iterate over the lines and search for the line that contains the following string: Successfully started service 'sparkExecutorActorSystem' on port.
        If the line is found, extract the port number and save it in the variable
        Make sure the varibles are scoped to be called in the main() function.
        If the line is not found,
        Return True along with the port number variable.
        Otherwise, return False.
        call the function passing the sample input
        Prompt 5: What is the sample input that can be tested as a test use case?
        {sample_input}

        prompt 6: Any additional expectation from the code logic?
        {additional_requirement}

        Prompt 7: Write the code for the scenario keeping additional expectation and expected language while generation along with the test case.

        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```c++
#include <iostream>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

// Function to identify the status of the sparkExecutorActorSystem service in network log
bool identifySparkExecutorActorSystemStatus(string networkLog) {
  // Split the network log into lines
  vector<string> lines;
  string line;
  while (getline(networkLog, line)) {
    lines.push_back(line);
  }

  // Iterate over the lines and search for the line that contains the following string: Successfully started service 'sparkExecutorActorSystem' on port.
  int port = -1;
  for (string line : lines) {
    if (line.find("Successfully started service 'sparkExecutorActorSystem' on port") != string::npos) {
      // Extract the port number and save it in the variable
      port = stoi(line.substr(line.find("port") + 5));
      break;
    }
  }

  // If the line is found, return True along with the port number
  if (port != -1) {
    return true;
  }

  // Otherwise, return False
  return false;
}

int main() {
  // Sample input network log
  string networkLog =
      "17/06/09 20:10:41 INFO slf4j.Slf4jLogger: Slf4jLogger started\n"
      "17/06/09 20:10:41 INFO Remoting: Starting remoting\n"
      "17/06/09 20:10:41 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkExecutorActorSystem@mesos-slave-07:55904]\n"
      "17/06/09 20:10:41 INFO util.Utils: Successfully started service 'sparkExecutorActorSystem' on port 55904.\n";

  // Call the function passing the sample input
  bool status = identifySparkExecutorActorSystemStatus(networkLog);

  // Print the output
  if (status) {
    cout << "True " << port << endl;
  } else {
    cout << "False" << endl;
  }

  return 0;
}
```

* **Java**

In [27]:
prefix = """
        q: What is the input to the function?
        a: The input to the function is a string.

        q: What is the output of the function?
        a: The output of the function is a reversed string.

        q: What are the steps involved in reversing a string?
        a: 1) Iterate over the string from the back.
            2) Add each character to a new string in reverse order.
            3) Return the new string.

        q: Write pseudocode for the function.
        a: function reverse_string(string):
              new_string = ""
              for i in range(len(string) - 1, -1, -1):
                  new_string += string[i]
              return new_string

        q: how would you test the function?
        a: the input "hello" should return "olleh"

        q: write java code for the function follwing all the question-answer pairs.
        """
response = code_generation_model.predict(prefix=prefix,max_output_tokens=2048).text

# you may use print instead of display
display(
    Markdown(response)
)

```java
import java.util.Scanner;

public class ReverseString {

    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);

        // Get the input string from the user.
        System.out.println("Enter a string: ");
        String inputString = scanner.nextLine();

        // Reverse the input string.
        String reversedString = reverseString(inputString);

        // Print the reversed string.
        System.out.println("The reversed string is: " + reversedString);
    }

    public static String reverseString(String string) {
        // Create a new string to store the reversed string.
        StringBuilder reversedString = new StringBuilder();

        // Iterate over the input string from the back.
        for (int i = string.length() - 1; i >= 0; i--) {
            // Add each character to the new string in reverse order.
            reversedString.append(string.charAt(i));
        }

        // Return the new string.
        return reversedString.toString();
    }
}
```

### 8. Few-shot with User-journey and Pseudo/Starter Code

In the few-shot prompt, we can also pass user-journey or pseudo/starter code examples for the code generation to adhere to specific instructions. The user journey can include sample input and output data structures as well. We can also complement that by passing some starter code so the model can follow the code generation per the structure we want. Here are some examples of different languages.

* **Kotlin**

In [28]:
user_journey = """
              A Kotlin developer is working on a new Android app.
              They need to implement a feature that allows users to search for nearby restaurants.
              """
sample_input = """
              A list of restaurants and a search query:
              val restaurants = listOf(
                                Restaurant("The Grill", "123 Main Street", cuisine = "American"),
                                Restaurant("Thai Paradise", "456 Elm Street", cuisine = "Thai"),
                                Restaurant("Little Italy", "789 Pine Street", cuisine = "Italian"),
                              )

              val searchQuery = "Italian"

              """
sample_output = """
          A list of restaurants that match the search query:
          val matchingRestaurants = listOf(
              Restaurant("Little Italy", "789 Pine Street", cuisine = "Italian"),
            )
          """
language = "Kotlin"
problem_statement = f"""a {language} function that takes a list of restaurants and a search query as input and returns  \
                        list of restaurants that match the search query. The search query can be a substring of the restaurant name, \
                        address, or cuisine type.
                    """
additional_requirement = """
                     - The function should be efficient and scalable.
                     - The function should be easy to read and maintain.
                      """
prefix = f"""
        Write {problem_statement}. \n
        Also add example use case that take {sample_input} and {sample_output} calling the function generated.
        Here are some additional requirement for the function: {additional_requirement}.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```kotlin
// Restaurant data class
data class Restaurant(val name: String, val address: String, val cuisine: String)

// Function to search for restaurants
fun searchRestaurants(restaurants: List<Restaurant>, searchQuery: String): List<Restaurant> {
    // Convert the search query to lowercase for case-insensitive search
    val lowercaseQuery = searchQuery.toLowerCase()

    // Filter the restaurants based on the search query
    return restaurants.filter { restaurant ->
        // Check if the restaurant name, address, or cuisine type contains the search query
        restaurant.name.toLowerCase().contains(lowercaseQuery) ||
                restaurant.address.toLowerCase().contains(lowercaseQuery) ||
                restaurant.cuisine.toLowerCase().contains(lowercaseQuery)
    }
}

// Example usage
val restaurants = listOf(
    Restaurant("The Grill", "123 Main Street", cuisine = "American"),
    Restaurant("Thai Paradise", "456 Elm Street", cuisine = "Thai"),
    Restaurant("Little Italy", "789 Pine Street", cuisine = "Italian"),
)

val searchQuery = "Italian"

val matchingRestaurants = searchRestaurants(restaurants, searchQuery)

// Print the matching restaurants
println("Matching restaurants:")
matchingRestaurants.forEach { restaurant ->
    println("${restaurant.name} - ${restaurant.address} - ${restaurant.cuisine}")
}
```

* **Rust**

In [29]:
real_world_case = """You are a software engineer at a company that develops trading software.
                     You need to write a Rust program to calculate the moving average of a stock price over a given period of time."""
problem_statement = f"""The program should take two inputs:
                        A vector of stock prices.
                        The period over which to calculate the moving average.
                        The program should output the moving average of the stock price over the given period.
                    """
code_style = "Idiomatic Rust"
algorithmic_complexity = "O(n)"
pseudocode = """
              1) Initialize a variable to store the moving average.
              2) Iterate over the vector of stock prices, adding each price to the moving average.
              3) Divide the moving average by the period to get the average price over the given period.
              4) Return the moving average price.
             """
test_cases = """
            // Test case 1
            let stock_prices = vec![100, 110, 120, 130, 140];
            let period = 3;

            let moving_average = calculate_moving_average(&stock_prices, period);

            assert_eq!(moving_average, 200.0);

            """
sample_input = "[500, 100, 300, 450, 120]"
prefix = f"""Write a Rust program based on a {real_world_case} .
             The problem statement that needs to be addressed is {problem_statement} .
             You can use this pseudocode as an example to generate the code step by step:  {pseudocode} .
             Add an example to call the generated function in main() {sample_input}
             It should follow the code style pattern as {code_style} and should have {algorithmic_complexity} as algorithmic complexity.
             Make sure that the code generated passes the following test cases: {test_cases}
             """
response = code_generation_model.predict(prefix=prefix,
                                         max_output_tokens=2000,
                                         temperature=0.2).text

# you may use print instead of display
display(
    Markdown(response)
)

```rust
// Calculate the moving average of a stock price over a given period of time.

// The function takes two inputs:
// - A vector of stock prices.
// - The period over which to calculate the moving average.

// The function returns the moving average of the stock price over the given period.

fn calculate_moving_average(stock_prices: &[f64], period: usize) -> f64 {
    // Initialize a variable to store the moving average.
    let mut moving_average = 0.0;

    // Iterate over the vector of stock prices, adding each price to the moving average.
    for i in 0..stock_prices.len() {
        moving_average += stock_prices[i];
    }

    // Divide the moving average by the period to get the average price over the given period.
    moving_average /= period as f64;

    // Return the moving average price.
    moving_average
}

fn main() {
    // Example usage of the calculate_moving_average function.
    let stock_prices = vec![500.0, 100.0, 300.0, 450.0, 120.0];
    let period = 3;

    let moving_average = calculate_moving_average(&stock_prices, period);

    println!("The moving average of the stock price over the given period is: {}", moving_average);
}
```

* **Scala**

In [30]:
persona = """You are a Scala developer working on a backend service for an e-commerce platform. """
goal = """Your task is to generate Scala code for a data model representing products in the platform's catalog.
          The product data model should include information such as product ID, name, price, and availability.
       """
user_journey = """
              As a developer, your day-to-day tasks often involve designing data models to represent various entities in your application.
              In this scenario, you are tasked with creating a Scala case class for the product data model and a companion object with utility methods.
              """
requirements = """
            1. Create a Scala case class named `Product` with the following fields:
                - `id` (String)
                - `name` (String)
                - `price` (Double)
                - `available` (Boolean)

            2. Implement a companion object for the `Product` case class with the following methods:
                - `create` method that takes parameters for ID, name, price, and availability and returns an instance of the `Product` case class.
                - `format` method that takes a `Product` instance and returns a formatted string representation of the product.

            3. Ensure that the `create` method sets the availability to `true` by default if not provided.
              """
code_structure = """
                You have been provided with a starter code structure. Your task is to complete the code to meet the above requirements. The input code structure is as follows:

                ```scala
                // Starter code
                case class Product(id: String, name: String, price: Double, available: Boolean)

                object Product {
                  // Your generated Scala code for the companion object goes here
                }

                object Main extends App {
                  // Your test cases go here
                }
                """

prefix = f"""{persona} {goal} {user_journey} {requirements} {code_structure} """
response = code_generation_model.predict(prefix=prefix,
                                         max_output_tokens=2000,
                                         temperature=0.2).text

# you may use print instead of display
display(
    Markdown(response)
)

```scala
// Starter code
case class Product(id: String, name: String, price: Double, available: Boolean)

object Product {
  // Companion object methods
  def create(id: String, name: String, price: Double, available: Boolean = true): Product = {
    Product(id, name, price, available)
  }

  def format(product: Product): String = {
    s"Product ID: ${product.id}, Name: ${product.name}, Price: ${product.price}, Available: ${product.available}"
  }
}

object Main extends App {
  // Test cases
  val product1 = Product.create("P1", "iPhone 13 Pro", 999.99, true)
  val product2 = Product.create("P2", "Samsung Galaxy S22 Ultra", 1099.99)

  println(Product.format(product1))
  println(Product.format(product2))
}
```

* **Shell Script**

In [31]:
persona = """
          Imagine you are a system administrator responsible for managing a Linux server environment.
          Your daily tasks often involve creating shell scripts to automate various system maintenance and monitoring tasks.
          In this scenario, you are tasked with generating a shell script to automate a common backup task.
          """
user_journey = """
          As a system administrator, you frequently need to create backup scripts to ensure data integrity and disaster recovery.
          Your goal is to generate a simple shell script that backs up a specified directory to a target backup location using the `rsync` command.
              """
requriements = """
          1. Create a shell script named `backup.sh` that takes two command-line arguments:
            - Source directory: The directory to be backed up.
            - Target directory: The directory where the backup should be stored.

          2. The script should use the `rsync` command to perform the backup. The `rsync` command should:
            - Synchronize the contents of the source directory to the target directory.
            - Preserve file permissions and timestamps.
            - Display progress information during the backup.

          3. Add comments to the script to explain its purpose and usage.
              """
starter_code = """
          You have been provided with a starter code structure.
          Your task is to complete the code to meet the above requirements.
          The input code structure is as follows:

          ```bash
          #!/bin/bash

          # Your generated Shell script code goes here
          """

prefix = f"""{persona} {user_journey} {requriements} {starter_code} """
response = code_generation_model.predict(prefix=prefix,
                                         max_output_tokens=2000,
                                        ).text

# you may use print instead of display
display(
    Markdown(response)
)

```bash
#!/bin/bash

# This script automates the process of backing up a specified directory to a target backup location using the rsync command.

# Check if the required number of arguments are provided
if [ $# -ne 2 ]; then
  echo "Usage: $0 <source_directory> <target_directory>"
  exit 1
fi

# Assign the command-line arguments to variables
source_dir=$1
target_dir=$2

# Check if the source directory exists
if [ ! -d "$source_dir" ]; then
  echo "Error: Source directory $source_dir does not exist."
  exit 1
fi

# Check if the target directory exists
if [ ! -d "$target_dir" ]; then
  echo "Error: Target directory $target_dir does not exist."
  exit 1
fi

# Perform the backup using rsync
rsync -avz --progress "$source_dir/" "$target_dir/"

# Check if the backup completed successfully
if [ $? -eq 0 ]; then
  echo "Backup completed successfully."
else
  echo "Error: Backup failed."
  exit 1
fi
```

* **Solidity [BlockChain]**

In [32]:
user_input = """
         - The address of the user withdrawing the tokens
         - The amount of tokens being withdrawn
         """
return_output = """
         - A boolean value indicating whether the withdrawal was successful
         """
requirements = """
          - The code should be written in Solidity using the latest best practices.
          - algorithm should be O(1) time complexity.
          """


prefix = f"""
         Generate a Solidity function called withdraw() that allows users to withdraw tokens from a decentralized exchange (DEX).
         The function should take the following inputs: {user_input} and should return: {return_output}.
         The function should also meet the following requirements: {requirements}
         """
response = code_generation_model.predict(prefix=prefix,
                                         max_output_tokens=2000,
                                         temperature=0.2).text

# you may use print instead of display
display(
    Markdown(response)
)

```solidity
pragma solidity ^0.8.0;

contract DEX {
    mapping(address => uint256) public balances;

    function withdraw(address user, uint256 amount) public returns (bool) {
        // Check if the user has enough tokens to withdraw
        require(balances[user] >= amount, "Insufficient balance");

        // Update the user's balance
        balances[user] -= amount;

        // Transfer the tokens to the user
        (bool success, ) = user.call{value: amount}("");
        require(success, "Transfer failed");

        return true;
    }
}
```

* **Verilog**

In [33]:
prefix = """
        Generate Verilog code for a 3-bit adder circuit that performs addition of two 3-bit numbers using only D flip-flops.
        The circuit should have the following inputs:
        a[2:0]
        b[2:0]
        The circuit should have the following outputs:
        sum[2:0]
        carry
        The circuit should implement the following logic:
        The circuit should add the two 3-bit numbers and store the result in the sum output. The carry output should be set to 1 if the addition results in a carry-out, and 0 otherwise.
        Constraints:
        The circuit must use only D flip-flops.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```verilog
module ThreeBitAdder(
    input [2:0] a,
    input [2:0] b,
    output [2:0] sum,
    output carry
);

// Instantiate D flip-flops
reg [2:0] sum_reg;
reg carry_reg;

// Combinational logic to calculate sum and carry
always @(*) begin
    {carry_reg, sum_reg} = a + b;
end

// Register the sum and carry values
always @(posedge clk) begin
    sum <= sum_reg;
    carry <= carry_reg;
end

endmodule
```

### 9. DevOps Templates

Codey is also capable of helping we generate DevOps-related code samples. We can generate `Docker`, `Jenkins`, `GitLab CI`, `Prometheus Configurations`, and many more such code blocks with the following templates.

* **Docker**

In [34]:
prefix = """
        Generate a Dockerfile for a Python application that:
          * Builds the image from a python:latest base image
          * Exposes the following ports: 8000
          * Installs the following dependencies: pip install flask
          * Sets the working directory to /app
          * Copies the following files to the image: app.py requirements.txt ./
          * Runs the following command on startup: flask run --host=0.0.0.0
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```
# Dockerfile for Python application

# Base image
FROM python:latest

# Expose ports
EXPOSE 8000

# Install dependencies
RUN pip install flask

# Set working directory
WORKDIR /app

# Copy files to image
COPY app.py requirements.txt ./ .

# Run command on startup
CMD flask run --host=0.0.0.0
```

* **Docker Compose**

In [35]:
prefix = """
        Generate a Compose file for the following services: web
          * Define the following networks: default
          * Define the following volumes: ./app:/app
          * Define the following environments: FLASK_APP=app.py
          * Define the following links: none
          * Define the following depends_on relationships: none
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```yaml
version: '3'

services:
  web:
    build: .
    volumes:
      - ./app:/app
    environment:
      - FLASK_APP=app.py
    networks:
      - default

networks:
  default:
```

* **Jenkins**

In [36]:
prefix = """
        Objective: Generate a Jenkinsfile for a parametrized pipeline for a Java project.
        Instructions:
        Allow the user to input parameters like "Environment" and "Feature Toggle."
        Use these parameters in the build and test stages.
        Provide default values for parameters.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```groovy
pipeline {
    parameters {
        choice(name: 'ENVIRONMENT', choices: ['dev', 'test', 'prod'], description: 'Select the environment to deploy to')
        booleanParam(name: 'FEATURE_TOGGLE', defaultValue: false, description: 'Enable or disable the feature toggle')
    }

    stages {
        stage('Build') {
            steps {
                echo "Building the application..."
                sh 'mvn clean package'
            }
        }

        stage('Test') {
            steps {
                echo "Running tests..."
                sh 'mvn test'
            }
        }

        stage('Deploy') {
            steps {
                echo "Deploying the application to ${params.ENVIRONMENT}..."
                sh "deploy.sh ${params.ENVIRONMENT}"
            }
        }
    }
}
```

* **GitLab CLI**

In [37]:
prefix = """
        Generate GitLab CI YAML configuration for a Node.js project.
        Instructions:
          - Assume the project uses Node.js and has unit tests.
          - Include stages for "Build," "Test," and "Deploy."
          - Specify Node.js version and test script.
          - Use GitLab CI variables for sensitive information.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```yaml
# GitLab CI/CD configuration for Node.js project

stages:
  - build
  - test
  - deploy

# Build stage
build:
  image: node:16
  script:
    - npm install
    - npm run build

# Test stage
test:
  image: node:16
  script:
    - npm test

# Deploy stage
deploy:
  image: nginx:1.21
  script:
    - cp -r build/* /usr/share/nginx/html
    - nginx -g "daemon off;"
```

* **Prometheus Configurations**

In [38]:
prefix = """
         Generate YAML configurations for Prometheus to monitor a containerized application.
          Instructions:
            - Specify the job for scraping metrics.
            - Include targets, labels, and metric relabeling as needed.
            - Set up alerting rules for key metrics.
        """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```yaml
# Prometheus configuration file for monitoring a containerized application

global:
  scrape_interval: 15s

scrape_configs:
  - job_name: 'my-app'
    static_configs:
      - targets: ['localhost:9090']
        labels:
          app: my-app
          env: production
    relabel_configs:
      - source_labels: [__meta_kubernetes_pod_container_name]
        action: keep
        regex: my-app
      - source_labels: [__meta_kubernetes_pod_name]
        action: keep
        regex: (.*)
        target_label: pod

alerting:
  alertmanagers:
    - static_configs:
        - targets:
            - localhost:9093

  rules:
    - alert: HighRequestLatency
      expr: sum(rate(http_request_duration_seconds_bucket{job="my-app"}[5m])) > 0.5
      for: 5m
      labels:
        severity: high
      annotations:
        summary: High request latency
        description: The average request latency is high.
    - alert: LowRequestVolume
      expr: sum(rate(http_requests_total{job="my-app"}[5m])) < 100
      for: 5m
      labels:
        severity: low
      annotations:
        summary: Low request volume
        description: The average request volume is low.
```

### 10. Web Templates

We can also generate HTML and CSS codes based on prompts. In the below example, we can see how we can mention various details of a simple HTML page and its characteristics.

* **HTML**

In [39]:
prefix = """write a html code for a simple page:
            - Has a button "click me" and it shows how many time user has hit that button as its counter.
            - Add style element that has page header as "My Page Counter Demo" in brown color
            - Everything should be displayed as 'center'.
            - The counter button should be blue by default and when clicked it should be red.
            - The counter value should be in green color.
            - The bottom of the page should display this message "Codey Generated this page" in bold and big font.
"""
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```html
<!DOCTYPE html>
<html>
<head>
  <title>My Page Counter Demo</title>
  <style>
    body {
      text-align: center;
    }

    h1 {
      color: brown;
    }

    #counter-button {
      background-color: blue;
      color: white;
      padding: 10px 20px;
      border: none;
      border-radius: 5px;
    }

    #counter-button:hover {
      background-color: red;
    }

    #counter-value {
      color: green;
      font-size: 24px;
    }

    footer {
      font-weight: bold;
      font-size: 24px;
    }
  </style>
</head>
<body>
  <h1>My Page Counter Demo</h1>

  <button id="counter-button" onclick="incrementCounter()">Click Me</button>

  <p id="counter-value">0</p>

  <footer>Codey Generated this page</footer>

  <script>
    let counter = 0;

    function incrementCounter() {
      counter++;

      document.getElementById("counter-value").innerHTML = counter;
    }
  </script>
</body>
</html>
```

In [40]:
prefix = """
        Generate a responsive HTML code using Tailwind CSS for a landing page of a new e-commerce website that sells clothes. The landing page should have the following sections:
         - A header with a logo, a navigation bar, and a search bar.
         - A hero section with a large image of a model wearing clothes from the website and a call to action button.
         - A featured products section with a grid of images of the best-selling products on the website.
         - A testimonial section with quotes from satisfied customers.
         - A footer with contact information and social media links.
       """
response = code_generation_model.predict(prefix=prefix).text

# you may use print instead of display
display(
    Markdown(response)
)

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>E-commerce Landing Page</title>
  <link href="https://unpkg.com/tailwindcss@^2/dist/tailwind.min.css" rel="stylesheet">
</head>
<body>
  <header class="bg-white shadow-md">
    <div class="container mx-auto flex justify-between items-center py-4">
      <div class="flex items-center">
        <a href="/" class="text-xl font-bold">Logo</a>
      </div>
      <nav class="hidden md:block">
        <ul class="flex items-center">
          <li><a href="#" class="px-4 py-2 text-gray-700 hover:text-gray-900">Home</a></li>
          <li><a href="#" class="px-4 py-2 text-gray-700 hover:text-gray-900">Shop</a></li>
          <li><a href="#" class="px-4 py-2 text-gray-700 hover:text-gray-900">About</a></li>
          <li><a href="#" class="px-4 py-2 text-gray-700 hover:text-gray-900">Contact</a></li>
        </ul>
      </nav>
      <div class="hidden md:block">
        <input type="text" placeholder="Search" class="border border-gray-300 rounded-md px-4 py-2">
      </div>
    </div>
  </header>

  <section class="hero bg-gray-100 py-16">
    <div class="container mx-auto flex items-center justify-center">
      <div class="text-center">
        <h1 class="text-4xl font-bold">Discover the latest fashion trends</h1>
        <p class="text-gray-700 mt-4">Shop our collection of clothes, shoes, and accessories.</p>
        <a href="#" class="btn btn-primary mt-8">Shop now</a>
      </div>
    </div>
  </section>

  <section class="featured-products bg-white py-16">
    <div class="container mx-auto">
      <h2 class="text-2xl font-bold text-center">Featured products</h2>
      <div class="grid grid-cols-1 md:grid-cols-3 gap-4 mt-8">
        <div class="product">
          <img src="images/product-1.jpg" alt="Product 1" class="w-full h-64 object-cover">
          <div class="p-4">
            <h3 class="text-lg font-bold">Product 1</h3>
            <p class="text-gray-700 mt-2">$100.00</p>
            <a href="#" class="btn btn-primary mt-4">Add to cart</a>
          </div>
        </div>
        <div class="product">
          <img src="images/product-2.jpg" alt="Product 2" class="w-full h-64 object-cover">
          <div class="p-4">
            <h3 class="text-lg font-bold">Product 2</h3>
            <p class="text-gray-700 mt-2">$200.00</p>
            <a href="#" class="btn btn-primary mt-4">Add to cart</a>
          </div>
        </div>
        <div class="product">
          <img src="images/product-3.jpg" alt="Product 3" class="w-full h-64 object-cover">
          <div class="p-4">
            <h3 class="text-lg font-bold">Product 3</h3>
            <p class="text-gray-700 mt-2">$300.00</p>
            <a href="#" class="btn btn-primary mt-4">Add to cart</a>
          </div>
        </div>
      </div>
    </div>
  </section

### Best Practices

**How to write effective code generation prompts**

When writing code generation prompts, it is important to be as clear and specific as possible. The more information we can provide to the model, the better it will be able to understand we intent and generate the desired code.

Here are some tips for writing effective code generation prompts:

* Start with a clear and concise description of the task we want the model to perform. For example, instead of saying _"Generate a function to sort a list of numbers,"_ we could say _"Generate a Python function to sort a list of integers in ascending order."_

* Provide examples of the desired input and output. This will help the model to understand the format of the data and the expected output. For example, _you could provide a list of unsorted numbers and the corresponding sorted list._

* Use natural language to describe the task. The model is trained on a massive dataset of text and code, so it is able to understand natural language prompts. For example, we could say _"Generate a function to reverse a string in Python."_

**How to choose the right temperature and max output tokens**

The temperature parameter controls the randomness of the model's output. _A higher temperature will result in more creative and varied output_, but it may also be less accurate. _A lower temperature will result in more accurate output_, but it may also be less creative.

The max output tokens parameter controls the maximum number of tokens that the model will generate. This is useful for limiting the length of the output code or preventing the model from generating infinite loops.

Here are some tips for choosing the right temperature and max output tokens:

* Use a lower temperature for tasks that require high accuracy, such as generating code for machine learning models.

* Use a higher temperature for tasks that require creativity, such as generating code for web applications or games.

* Use a max output tokens parameter to limit the length of the output code or prevent the model from generating infinite loops.

How to interpret and use code generation suggestions

**_The code generation suggestions generated by the model are not always perfect_**. It is important to review the generated code carefully and make any necessary changes.

Here are some tips for interpreting and using code generation suggestions:

* Check the output code for any syntax errors.
* Make sure that the output code is consistent with our coding standards.
* Test the output code to make sure that it works as expected.

**How to avoid common code generation pitfalls**

Here are some common code generation pitfalls to avoid:

* Using ambiguous or unclear prompts. The more specific you can be in your prompts, the better the model will be able to understand your intent and generate the desired code.
* Using too high of a temperature. A higher temperature can lead to less accurate and more creative output. It is important to choose the right temperature for the task you are trying to perform.
* Not reviewing the generated code carefully. The generated code is not always perfect. It is important to review the generated code carefully and make any necessary changes.